In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'regression_annual_dataset.parquet'

In [ ]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f"Shape: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
df.head()

In [ ]:
# Info general
print("=" * 60)
print("INFO GENERAL")
print("=" * 60)
print(f"\nTotal registros: {len(df):,}")
print(f"Período: {df['anio'].min()} - {df['anio'].max()}")
print(f"Municipios: {df['codigo_municipio'].nunique()}")
print(f"\nValores nulos:")
nulls = df.isnull().sum()
print(nulls[nulls > 0] if any(nulls > 0) else "Ninguno")

## 1. Distribución de Targets

In [ ]:
# Identificar targets (columnas de tasas + total_delitos)
target_cols = ['total_delitos'] + [c for c in df.columns if c.startswith('tasa_')]
print("Targets identificados:")
for t in target_cols:
    print(f"  - {t}")

# Estadísticas de targets
print("\n" + "=" * 60)
print("ESTADÍSTICAS DE TARGETS")
print("=" * 60)
df[target_cols].describe().round(2)

In [ ]:
# Distribución de total_delitos
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Histograma
axes[0].hist(df['total_delitos'], bins=50, color='steelblue', edgecolor='black')
axes[0].set_xlabel('Total Delitos')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de total_delitos')

# Log-transform
df_log = np.log1p(df['total_delitos'])
axes[1].hist(df_log, bins=50, color='coral', edgecolor='black')
axes[1].set_xlabel('log(Total Delitos + 1)')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución log-transformada')

# Boxplot
axes[2].boxplot(df['total_delitos'].dropna())
axes[2].set_ylabel('Total Delitos')
axes[2].set_title('Boxplot total_delitos')

plt.tight_layout()
plt.show()

# Test de normalidad
_, p_value = stats.normaltest(df['total_delitos'].dropna())
print(f"\nTest de normalidad (D'Agostino): p-value = {p_value:.2e}")
print("⚠️ Distribución NO normal" if p_value < 0.05 else "✅ Distribución aproximadamente normal")
print("\nRecomendación: Considerar log-transform o modelos robustos a outliers")

In [ ]:
# Distribución de tasas
tasa_cols = [c for c in df.columns if c.startswith('tasa_')]

if tasa_cols:
    n_cols = min(4, len(tasa_cols))
    n_rows = (len(tasa_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 3*n_rows))
    axes = axes.flat if n_rows > 1 or n_cols > 1 else [axes]
    
    for ax, col in zip(axes, tasa_cols):
        df[col].hist(bins=30, ax=ax, color='steelblue', edgecolor='black')
        ax.set_title(col.replace('tasa_', '').title())
        ax.set_xlabel('Tasa')
    
    # Ocultar ejes vacíos
    for ax in axes[len(tasa_cols):]:
        ax.set_visible(False)
    
    plt.suptitle('Distribución de Tasas por Delito', y=1.02)
    plt.tight_layout()
    plt.show()

## 2. Análisis Temporal

In [ ]:
# Evolución temporal de delitos
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Total delitos por año
delitos_por_anio = df.groupby('anio')['total_delitos'].sum()
axes[0].plot(delitos_por_anio.index, delitos_por_anio.values, marker='o', linewidth=2)
axes[0].set_xlabel('Año')
axes[0].set_ylabel('Total Delitos')
axes[0].set_title('Total Delitos por Año (Departamento)')
axes[0].grid(True)

# Promedio por municipio por año
promedio_por_anio = df.groupby('anio')['total_delitos'].mean()
axes[1].bar(promedio_por_anio.index, promedio_por_anio.values, color='coral')
axes[1].set_xlabel('Año')
axes[1].set_ylabel('Promedio Delitos')
axes[1].set_title('Promedio Delitos por Municipio por Año')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3. Correlaciones

In [ ]:
# Seleccionar features numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Matriz de correlación
fig, ax = plt.subplots(figsize=(14, 12))
corr_matrix = df[numeric_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, ax=ax, annot_kws={'size': 7})
ax.set_title('Matriz de Correlación')
plt.tight_layout()
plt.show()

In [ ]:
# Correlaciones con total_delitos
print("=" * 60)
print("CORRELACIONES CON total_delitos")
print("=" * 60)

corr_target = df[numeric_cols].corr()['total_delitos'].drop('total_delitos').sort_values(ascending=False)
print("\nTop 10 correlaciones positivas:")
print(corr_target.head(10).round(3))
print("\nTop 10 correlaciones negativas:")
print(corr_target.tail(10).round(3))

## 4. Análisis de Outliers

In [ ]:
# Detección de outliers con IQR
print("=" * 60)
print("ANÁLISIS DE OUTLIERS (total_delitos)")
print("=" * 60)

Q1 = df['total_delitos'].quantile(0.25)
Q3 = df['total_delitos'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['total_delitos'] < lower_bound) | (df['total_delitos'] > upper_bound)]

print(f"\nQ1: {Q1:.0f}")
print(f"Q3: {Q3:.0f}")
print(f"IQR: {IQR:.0f}")
print(f"Límite inferior: {lower_bound:.0f}")
print(f"Límite superior: {upper_bound:.0f}")
print(f"\nOutliers detectados: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")

if len(outliers) > 0:
    print(f"\nTop 10 outliers:")
    cols_show = ['codigo_municipio', 'anio', 'total_delitos', 'poblacion_total']
    cols_show = [c for c in cols_show if c in df.columns]
    print(outliers.nlargest(10, 'total_delitos')[cols_show])

## 5. Conclusiones y Recomendaciones

In [ ]:
print("=" * 60)
print("CONCLUSIONES")
print("=" * 60)

skewness = df['total_delitos'].skew()
kurtosis = df['total_delitos'].kurtosis()

print(f"""
DISTRIBUCIÓN DEL TARGET (total_delitos):
  - Media: {df['total_delitos'].mean():.0f}
  - Mediana: {df['total_delitos'].median():.0f}
  - Std: {df['total_delitos'].std():.0f}
  - Skewness: {skewness:.2f} ({'Alta asimetría positiva' if skewness > 1 else 'Moderada'})
  - Kurtosis: {kurtosis:.2f}
  - Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)

DATASET:
  - Registros: {len(df):,}
  - Features: {len(df.columns)}
  - Período: {df['anio'].min()} - {df['anio'].max()}
""")

print("\n" + "=" * 60)
print("RECOMENDACIONES PARA MODELO")
print("=" * 60)
print("""
1. TRANSFORMACIÓN DEL TARGET:
   - Aplicar log-transform: y = log(total_delitos + 1)
   - O usar modelos robustos a outliers

2. MODELOS SUGERIDOS:
   - XGBoost / LightGBM (robustos a outliers)
   - Random Forest
   - Elastic Net (si se normaliza)

3. MÉTRICAS:
   - RMSE, MAE (en escala original)
   - MAPE para interpretabilidad
   - R² para varianza explicada

4. VALIDACIÓN:
   - TimeSeriesSplit por año
   - O GroupKFold por municipio

5. FEATURES:
   - Lag features del año anterior
   - Tendencia temporal
   - Interacciones población × área
""")